In [9]:
import arcpy
arcpy.env.overwriteOutput = True 
from arcpy.sa import *
from arcpy import env

import os
import pandas as pd
import numpy as np

workspace = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files', 'Flo_py_wrkspace4')
tempspace = os.path.join(workspace, "temp")

outspace = os.path.join(workspace, 'Outputs')
#make sure workspace directory exists
if not os.path.exists(outspace):
    os.makedirs(outspace)

In [2]:
# Find areas where infrastructure is seated below a certain elevation on a DEM   ( Not sure if I need thiss?? # arcpy.CheckOutExtension("Spatial")) 

SLR_level = 3    # the level of sea level rise to stipulate

# make a 1 or 0 raster of the areas where the raster value is LT or GT the stipliated value (have to pay attention the sign 
in_raster = os.path.join("..",  'Data/Raw/GIS/3m_DEM', '3m_dem_cp.tif')
outraster = os.path.join(tempspace, 'LT1m1mdem.tif')
DaNewraster = Raster(in_raster) < SLR_level  
DaNewraster.save(outraster)

# Convert the raster to polygons
outPolygons = os.path.join(tempspace, 'LT1m1mdem_polys2.shp')
arcpy.RasterToPolygon_conversion(outraster, outPolygons, "NO_SIMPLIFY")

# Delete any areas where the raster (now gridcode attribute) is 0
criteria =  "gridcode = 0"  
arcpy.MakeFeatureLayer_management(outPolygons, "Temp_Poly", criteria)   
arcpy.DeleteRows_management("Temp_Poly")  


<Result 'Temp_Poly'>

In [8]:
#  Clip some infrastructure based on the inundated areas
intoclip = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped', "Roads_2006_clip.shp" ) 

out_clipped  = os.path.join(tempspace, "TESTING_3m_Roads_FLOODED.shp")
arcpy.Clip_analysis(intoclip, os.path.join(tempspace, 'LT1m1mdem_polys2.shp'), out_clipped)

<Result '..\\..\\RISA_yr6_GIT_ignored_files\\Flo_py_wrkspace4\\temp\\TESTING_3m_Roads_FLOODED.shp'>

In [14]:
# Clip all infrastructures in a folder

InfraShpPath = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped')

for shpo in os.listdir(InfraShpPath):
    if shpo.endswith(".shp"):
        intoclip = os.path.join(InfraShpPath, shpo ) 
       
        out_clipped  = os.path.join(outspace, "{}_FLOODED.shp".format(shpo))
        arcpy.Clip_analysis(intoclip, os.path.join(tempspace, 'LT1m1mdem_polys2.shp'), out_clipped)     
   


In [19]:
boop  =os.path.join(outspace, "{}_FLOODED.shp".format(shpo)) 
arcpy.management.CalculateGeometryAttributes_management(boop, "LENGTH")

AttributeError: module 'arcpy.management' has no attribute 'CalculateGeometryAttributes_management'